In [1]:
from PIL import Image
import pytesseract
import requests
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import pypandoc
import telebot
from telebot import types
import docx2txt
import os
import shutil

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    converter.close()
    fake_file_handle.close()

    if text:
        return text

In [4]:
def word2pdf (input_path):
    pypandoc.convert_file(input_path,
                      'pdf',
                      outputfile=input_path.split('.')[0] + '.pdf',
                      extra_args=[
                          '-V', 'geometry:margin=1.5cm', '--pdf-engine',
                          'C:\\texlive\\2019\\bin\\win32\\xelatex.exe'
                      ])
    return input_path.split('.')[0] + '.pdf'

In [5]:
need_convert = False
def globallyChange(x=None):
    global need_convert
    if x!= None:
        need_convert = x
    return need_convert

In [23]:
API_TOKEN = ''
bot = telebot.TeleBot(API_TOKEN)

In [25]:
@bot.message_handler(content_types=['photo', 'document', 'text'])
def get_text_messages(message):
    if message.content_type == 'photo':
        bot.send_message(
            message.from_user.id,
            "Твое изображение получил, сейчас обработаю и скину текст. Местами могу ошибаться, я еще только учусь."
        )
        a = message.json
        file_id = a['photo'][-1]['file_id']
        file_info = bot.get_file(file_id)

        file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(
            API_TOKEN, file_info.file_path))

        with open('C:/Users/Fedor/Documents/Edu/ComputerNetworks/file.jpg',
                  'wb') as f:
            f.write(file.content)

        bot.send_message(message.from_user.id, "Так, картинка уже у меня...")
        file = Image.open(
            'C:/Users/Fedor/Documents/Edu/ComputerNetworks/file.jpg')
        im = file.convert('L')
        text = pytesseract.image_to_string(im, lang=['rus'])
        print(text)
        bot.send_message(message.from_user.id, text)

    if message.content_type == 'document':
        a = message.json
        ext = str(a['document']['file_name']).split('.')[-1]
        
        if ext == 'pdf':
            bot.send_message(message.from_user.id,
                             "Твой pdf получил, сейчас обработаю...")

            file_id = a['document']['file_id']
            file_info = bot.get_file(file_id)
            file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(
                API_TOKEN, file_info.file_path))
            path = 'C:/Users/Fedor/Documents/Edu/ComputerNetworks/' + str(
                a['document']['file_name'])
            with open(path, 'wb') as f:
                f.write(file.content)

            bot.send_message(message.from_user.id, "Так, уже скачал...")
            text = extract_text_from_pdf(path)
            bot.send_message(message.from_user.id, text)
            print(text)                           
                
        if ext == 'doc' or ext == 'docx':           
            bot.send_message(message.from_user.id,
                 "Твой документ получил, сейчас обработаю...")
            
            file_id = a['document']['file_id']
            file_info = bot.get_file(file_id)
            file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(
                API_TOKEN, file_info.file_path))
            path = 'C:/Users/Fedor/Documents/Edu/ComputerNetworks/' + str(
                a['document']['file_name'])
            with open(path, 'wb') as f:
                f.write(file.content)
                
            if globallyChange():
                # Пытаемся конвертировать в pdf
                try:
                    output = word2pdf(path)
                except Exception:
                    bot.send_message(message.from_user.id, "Не умею работать с этим форматом :(") 
                    
                # Отправляем pdf
                doc = open(output, 'rb')
                bot.send_message(message.from_user.id, "Твой файл в формате pdf:") 
                bot.send_document(message.from_user.id, doc)
                globallyChange(False)
            else:    
                
                # Отправляем текст из документа
                path_to_images = "C:\\Users\\Fedor\\Documents\\Edu\\ComputerNetworks\\images"
                text = docx2txt.process(path, path_to_images)
                bot.send_message(message.from_user.id, "Так, ну вот текст из документа:")            
                bot.send_message(message.from_user.id, text)
                
                # Отправляем изображения из документа                
                images = [os.path.join(path_to_images,f) for f in os.listdir(path_to_images)]                
                bot.send_message(message.from_user.id, "Изображения из документа:")                
                for image in images:
                    im = open(image, 'rb')
                    bot.send_photo(message.from_user.id, im)
                    im.close()
                list(map(lambda x: os.remove(x), images))
                
        else:
            bot.send_message(message.from_user.id,
                 "Твой документ получил, сейчас обработаю...")
            
            file_id = a['document']['file_id']
            file_info = bot.get_file(file_id)
            file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(
                API_TOKEN, file_info.file_path))
            path = 'C:/Users/Fedor/Documents/Edu/ComputerNetworks/' + str(
                a['document']['file_name'])
            with open(path, 'wb') as f:
                f.write(file.content)
            
            if need_convert:
                # Пытаемся конвертировать в pdf
                try:
                    output = word2pdf(path)
                except Exception:
                    bot.send_message(message.from_user.id, "Не умею работать с этим форматом :(")                
            else:
                bot.send_message(message.from_user.id, "Такой формат файла могу только конвертировать в pdf. Для этого нажми на кнопку /Convert to pdf")
                
                
    if message.content_type == 'text':
        if message.text == "/command1":
            bot.send_message(
                message.from_user.id,
                'Привет еще раз! Просто отправь мне файл формата PDF или обычный скриншот, я постараюсь вытащить оттуда текст :)'
            )
            
        if message.text == "/command2":
            bot.send_message(
                message.from_user.id,
                "Я самый обычный телеграмм-бот, использую технологии компьютерного зрения для обработки фотографий. Мой код лежит тут: *ссылка на гит*"
            )
            
        if message.text == "/start":
            bot.send_message(
                message.from_user.id,
                "Привет, {0}! Меня зовут TextGrabber. Я помогу тебе достать текст с картинки или PDF файла. Нажми /command1"
                .format(str(message.from_user.first_name)))
            
            markup = types.ReplyKeyboardMarkup()
            itembtna = types.KeyboardButton('Convert to pdf')
            markup.row(itembtna)
            bot.send_message(message.from_user.id, "", reply_markup=markup)
            globallyChange(False)

        if message.text == "Convert to pdf":           
            globallyChange(True)
            bot.send_message(
                message.from_user.id,
                'Жду твой документ...' + str(globallyChange())
            )        
    
    
bot.polling(none_stop=True, interval=0)
